# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1o2lpxoi9heyQV1hIlsHXWSfDkBPtze-V?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [1]:
# instalar o Selenium -> no prompt, pip install selenium.
# google chrome -> chrome driver.

from selenium import webdriver
from selenium.webdriver.common.keys import Keys

#abrir um navegador
navegador = webdriver.Chrome()

navegador.get('https://www.google.com.br/')

#agora preciso pegar a cotação do dólar:

#na pagina web, botão direito do mouse, inspecionar, setinha no canto esquerdo superior da pagina de códigos que abriu.
#depois, botão direito do mouse em cima da caixa de escrever do google e Copy > Copy XPath.
#cola dentro desse parenteses aí embaixo, depois digite .send_keys e coloque entre parenteses o que deseja procurar no Google.

navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação dólar')
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
cotacao_dolar = navegador.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cotacao_dolar)
navegador.quit()

#agora preciso pegar a cotação do Euro:

navegador = webdriver.Chrome()
navegador.get('https://www.google.com.br/')
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação euro')
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
cotacao_euro = navegador.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cotacao_euro)
navegador.quit()

#agora preciso pegar a cotação do Ouro:

navegador = webdriver.Chrome()
navegador.get('https://www.melhorcambio.com/ouro-hoje')
cotacao_ouro = navegador.find_element_by_xpath('//*[@id="comercial"]').get_attribute('value')

#trocar a vírgula que está na cotação do ouro por ponto pois o PYTHON só entende ponto e não entende vírgula.

cotacao_ouro = cotacao_ouro.replace(',', '.')
print(cotacao_ouro)
navegador.quit()


5.231445
6.193769307
302.37


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [2]:
# Passo 4: Importar a lista de produtos da empresa
import pandas as pd

tabela = pd.read_excel('Produtos.xlsx')
display(tabela)



,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Margem,Preço Final
0,Câmera Canon,999.99,Dólar,5.0405,5040.449595,1.40,7056.629433
1,Carro Renault,4500.00,Euro,5.9685,26858.250000,2.00,53716.500000
2,Notebook Dell,899.99,Dólar,5.0405,4536.399595,1.70,7711.879311
3,IPhone,799.00,Dólar,5.0405,4027.359500,1.70,6846.511150
4,Carro Fiat,3000.00,Euro,5.9685,17905.500000,1.90,34020.450000
5,Celular Xiaomi,480.48,Dólar,5.0405,2421.859440,2.00,4843.718880
6,Joia 20g,20.00,Ouro,288.8100,5776.200000,1.15,6642.630000


- Atualizando os preços e o cálculo do Preço Final

In [3]:
# Passo 5: Recalcular o preço de cada produto
# -> atualizar a cotação
# -> nas linhas onde na coluna 'MOEDA' = 'DÓLAR'

tabela.loc[tabela['Moeda']=='Dólar', 'Cotação'] = float(cotacao_dolar)
tabela.loc[tabela['Moeda']=='Euro', 'Cotação'] = float(cotacao_euro)
tabela.loc[tabela['Moeda']=='Ouro', 'Cotação'] = float(cotacao_ouro)

# -> atualizar o PREÇO BASE REAIS (Preço Base Original * Cotação)
tabela['Preço Base Reais'] = tabela['Preço Base Original'] * tabela['Cotação']

# -> atualizar o PREÇO FINAL (Preço Base Reais * Mergem)
tabela['Preço Final'] = tabela['Preço Base Reais'] * tabela['Margem']

display(tabela)




,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Margem,Preço Final
0,Câmera Canon,999.99,Dólar,5.231445,5231.392686,1.40,7323.949760
1,Carro Renault,4500.00,Euro,6.193769,27871.961881,2.00,55743.923763
2,Notebook Dell,899.99,Dólar,5.231445,4708.248186,1.70,8004.021915
3,IPhone,799.00,Dólar,5.231445,4179.924555,1.70,7105.871743
4,Carro Fiat,3000.00,Euro,6.193769,18581.307921,1.90,35304.485050
5,Celular Xiaomi,480.48,Dólar,5.231445,2513.604694,2.00,5027.209387
6,Joia 20g,20.00,Ouro,302.370000,6047.400000,1.15,6954.510000


### Agora vamos exportar a nova base de preços atualizada

In [4]:
# Passo 6: Salvar os novos preços dos produtos
# index = False , significa que eu não quero que os números no inicio ao lado direito da minha tabela estejam aparecendo.
tabela.to_excel('Produtos2.xlsx', index=False)
